# imagenator Regression Tests

This notebook runs a suite of regression tests against the Harmony imagenator Service chain.
These tests use a sample TEMPO NO₂ granule to verify that imagenator logic works as expected and that output matches reference data in `reference_data/`.


In [ ]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'
# (Overridden if you set HARMONY_HOST_URL in the shell before running.)

## Prerequisites

- Create the `papermill-imagenator` environment:
  ```bash
  conda env create -f ./environment.yaml && conda activate papermill-imagenator
  ```
- Ensure a `.netrc` file with your Earthdata Login credentials is present here.


### Import required packages:


In [ ]:
from harmony import Client, Collection, Environment, Request
from utilities import validate_imagenator_outputs

### Set up environment-dependent variables for imagenator L2:

Define the collection and granule for testing in each environment.

In [ ]:
# TEMPO NO2 Level-2 data
non_production_collection = {
    'filter_collection': Collection(id='C1262899916-LARC_CLOUD')
}
non_prod_granule_data = {'filter_granules': ['G1269044683-LARC_CLOUD']}

collection_data = {
    'https://harmony.uat.earthdata.nasa.gov': {
        **non_production_collection,
        **non_prod_granule_data,
        'env': Environment.UAT,
    },
    'https://harmony.sit.earthdata.nasa.gov': {
        **non_production_collection,
        **non_prod_granule_data,
        'env': Environment.SIT,
    },
    'http://localhost:3000': {
        **non_production_collection,
        **non_prod_granule_data,
        'env': Environment.LOCAL,
    },
}

environment_information = collection_data.get(harmony_host_url)

if environment_information is not None:
    harmony_client = Client(env=environment_information['env'])
    filter_collection = environment_information['filter_collection']
    filter_granule = environment_information['filter_granules'][0]
else:
    print('Skipping tests: imagenator L2 is not configured for environment')

## Test: imagenator L2 request on TEMPO NO2 vertical_column_stratosphere variable

Submit a single-granule request and validate output files.

In [ ]:
if environment_information is not None:
    # Build the request
    request = Request(collection=filter_collection, granule_id=[filter_granule],
                      variables='product/vertical_column_stratosphere',format='image/png')
    print(harmony_client.request_as_curl(request))

    # Submit and wait for completion
    job_id = harmony_client.submit(request)
    harmony_client.wait_for_processing(job_id, show_progress=True)

    # Validate against reference_data/
    validate_imagenator_outputs(harmony_client, job_id)
else:
    print('Skipping test: imagenator L2 not configured for environment')

### Set up environment-dependent variables for imagenator L3:

Define the collection and granule for testing in each environment.

In [ ]:
# TEMPO NO2 Level-3 data
non_production_collection = {
    'filter_collection': Collection(id='C1262899964-LARC_CLOUD')
}
non_prod_granule_data = {'filter_granules': ['G1269043414-LARC_CLOUD']}

collection_data = {
    'https://harmony.uat.earthdata.nasa.gov': {
        **non_production_collection,
        **non_prod_granule_data,
        'env': Environment.UAT,
    },
    'https://harmony.sit.earthdata.nasa.gov': {
        **non_production_collection,
        **non_prod_granule_data,
        'env': Environment.SIT,
    },
    'http://localhost:3000': {
        **non_production_collection,
        **non_prod_granule_data,
        'env': Environment.LOCAL,
    },
}

environment_information = collection_data.get(harmony_host_url)

if environment_information is not None:
    harmony_client = Client(env=environment_information['env'])
    filter_collection = environment_information['filter_collection']
    filter_granule = environment_information['filter_granules'][0]
else:
    print('Skipping tests: imagenator L3 is not configured for environment')

## Test: imagenator L3 request on TEMPO NO2 vertical_column_stratosphere variable

Submit a single-granule request and validate output files.

In [ ]:
if environment_information is not None:
    # Build the request
    request = Request(collection=filter_collection, granule_id=[filter_granule],
                      variables='product/vertical_column_stratosphere',format='image/png')
    print(harmony_client.request_as_curl(request))

    # Submit and wait for completion
    job_id = harmony_client.submit(request)
    harmony_client.wait_for_processing(job_id, show_progress=True)

    # Validate against reference_data/
    validate_imagenator_outputs(harmony_client, job_id)
else:
    print('Skipping test: imagenator L3 not configured for environment')